<a href="https://colab.research.google.com/github/eriksali/DNN_2023_DL/blob/main/Input-dependent-certification_random_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl -O https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar xzf imagenette2.tgz
import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1485M  100 1485M    0     0  48.4M      0  0:00:30  0:00:30 --:--:-- 47.5M


In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define dataset and data loader
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

dataset = ImageNet(root='./imagenette2', split='train', transform=data_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train a ResNet50 model on ImageNet with Gaussian data augmentation
model = models.resnet50(pretrained=False).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

def train_model(model, optimizer, scheduler, dataloader, epochs=100, sigma=0.1):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)

            # Generate Gaussian noise
            noise = sigma * torch.randn_like(inputs).to(device)

            # Add noise to inputs
            inputs_noisy = inputs + noise

            # Compute logits
            logits = model(inputs_noisy)

            # Compute loss
            loss = torch.nn.functional.cross_entropy(logits, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            predicted = torch.argmax(logits, dim=1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

            if i % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}, Accuracy: {100*correct/total:.2f}%")

        # Update learning rate
        scheduler.step()

    # Return the trained model
    return model

# Train the model
model = train_model(model, optimizer, scheduler, dataloader, epochs=10, sigma=0.1)

# Define the smoothed classifier g
def smoothed_classifier(x, model, n_samples, sigma):
    model.eval()
    with torch.no_grad():
        logits = torch.zeros((n_samples,) + model(x).shape).to(device)
        for i in range(n_samples):
            noise = sigma * torch.randn_like(x).to(device)
            logits[i] = model(x + noise)
        avg_logits = torch.mean(logits, dim=0)
        return avg_logits

# Test the smoothed classifier on an example image
from PIL import Image
import requests
from io import BytesIO

url = 'https://upload.wikimedia.org/wikipedia/commons/3/32/Tom_Cruise_by_Gage_Skidmore.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = data_transform(img).unsqueeze(0).to(device)

n_samples = 100
sigma = 0.1

logits = smoothed_classifier(img, model, n_samples, sigma)
pred_label = torch.argmax(logits).item()

print(f"Predicted class: {pred_label}")


RuntimeError: ignored

In [ ]:
!curl -O https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar xzf imagenette2.tgz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1485M  100 1485M    0     0  42.1M      0  0:00:35  0:00:35 --:--:-- 41.3M


In [ ]:
import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')


In [ ]:

'''
This code will evaluate the smoothed classifier on the test set and print out the test accuracy. 
Note that this may take some time to run, depending on the size of the test set and the number of samples used for smoothing.
'''


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Gaussian data augmentation function
def gaussian_noise(image, sigma):
    return image + torch.randn_like(image) * sigma

# Define the dataset class for ImageNet
class ImageNetDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.dataset = torchvision.datasets.ImageFolder(root=root, transform=transform)
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        image = gaussian_noise(image, sigma)
        return image, label
    
    def __len__(self):
        return len(self.dataset)

'''import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')'''

# Load the ImageNet dataset
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])
train_dataset = ImageNetDataset(root='imagenette2/train', transform=transform)
test_dataset = ImageNetDataset(root='imagenette2/val', transform=transform)

# Define the neural network f and train it
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(64*56*56, 512)
        self.fc2 = torch.nn.Linear(512, 1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}")

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

sigma = 0.1
n_samples = 100



# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenet', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


In [ ]:
!git clone https://github.com/fastai/imagenette
!pip install torchvision
import tarfile

with tarfile.open('/content/imagenette/imagenette2-160.tgz', 'r:gz') as tar:
    tar.extractall('/content/named_folders/imagenet-100')


fatal: destination path 'imagenette' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


FileNotFoundError: ignored

In [ ]:
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_train.tar
!tar -xvf ILSVRC2012_img_train.tar

!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar
!tar -xvf ILSVRC2012_img_val.tar

import torchvision.datasets as datasets
import os

!mkdir imagenet
# Define the path to the ImageNet data
data_path = './imagenet/'

# Define the subset of classes to use
classes = ['cat', 'dog', 'bird']

# Create a PyTorch dataset using the ImageFolder class
dataset = datasets.ImageFolder(root=data_path, 
                                transform=transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])
                                ]), 
                                target_transform=lambda x: classes.index(os.path.basename(os.path.dirname(x))))

# Split the dataset into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len(dataset)-500, 500])


In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the ImageNet dataset
train_dataset = datasets.ImageNet(root='imagenet', split='train', transform=transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))

# Download the dataset
train_dataset.download()


In [ ]:

'''
This code will evaluate the smoothed classifier on the test set and print out the test accuracy. 
Note that this may take some time to run, depending on the size of the test set and the number of samples used for smoothing.
'''


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Gaussian data augmentation function
def gaussian_noise(image, sigma):
    return image + torch.randn_like(image) * sigma

# Define the dataset class for ImageNet
class ImageNetDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.dataset = torchvision.datasets.ImageFolder(root=root, transform=transform)
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        image = gaussian_noise(image, sigma)
        return image, label
    
    def __len__(self):
        return len(self.dataset)

# Load the ImageNet dataset
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])
train_dataset = ImageNetDataset(root='path/to/imagenet/train', transform=transform)
test_dataset = ImageNetDataset(root='path/to/imagenet/val', transform=transform)

# Define the neural network f and train it
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(64*56*56, 512)
        self.fc2 = torch.nn.Linear(512, 1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}")

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

sigma = 0.1
n_samples = 100



# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenet', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


In [ ]:
!pip install torch torchvision

In [ ]:


#!pip install torch torchvision

import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True).to(device)
model.eval()


import numpy as np
import torch.nn.functional as F

def randomized_smoothing(x, model, n_samples, noise_std):
    logits = torch.zeros((n_samples,) + model(x).shape).to(device)
    for i in range(n_samples):
        noise = noise_std * torch.randn(x.shape).to(device)
        logits[i] = model(x + noise)
    avg_logits = torch.mean(logits, dim=0)
    return F.softmax(avg_logits, dim=-1)


from PIL import Image
import requests
from io import BytesIO

url = 'https://upload.wikimedia.org/wikipedia/commons/3/32/Tom_Cruise_by_Gage_Skidmore.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = img.resize((224, 224), resample=Image.BILINEAR)

x = torch.tensor(np.array(img)).permute(2, 0, 1).unsqueeze(0).float().div(255).to(device)


n_samples = 100
noise_std = 0.1

y_smooth = randomized_smoothing(x, model, n_samples, noise_std)
pred_label = torch.argmax(y_smooth).item()

print(f"Predicted class: {pred_label}")


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import torch.nn.functional as F

# Define the standard deviation of the Gaussian noise
sigma = 0.1

# Define the number of samples for randomized smoothing
n_samples = 50

# Define the batch size for evaluation
batch_size = 64

# Define the device to run the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the ImageNet dataset with data augmentation
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
trainset = torchvision.datasets.ImageFolder(root='./imagenet/train', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# Train a ResNet50 model on the dataset
model = models.resnet50(pretrained=False).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
for epoch in range(10):
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Define the randomized smoothing function
def smoothed_classifier(x, model, n_samples, sigma):
    logits = torch.zeros((n_samples,) + model(x).shape).to(device)
    for i in range(n_samples):
        noise = sigma * torch.randn(x.shape).to(device)
        logits[i] = model(x + noise)
    avg_logits = torch.mean(logits, dim=0)
    return F.softmax(avg_logits, dim=-1)

# Evaluate the smoothed classifier on the test set
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
testset = torchvision.datasets.ImageFolder(root='./imagenet/val', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

total, correct = 0, 0
for i, (inputs, labels) in enumerate(testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = smoothed_classifier(inputs, model, n_samples, sigma)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print('Accuracy of the smoothed classifier on the test set: %.2f%%' % accuracy)
